# Nissen et al., 2023: Severe 21st-century OA in Antarctic MPAs
#
# script to save regional averages of bottom fields of omega aragonite, omega calcite and pH as netcdf files
# Antarctic Peninsula only!
# requires full model output
# note that some paths are hard-coded throughout the script

In [1]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata
from matplotlib import cm
from netCDF4 import Dataset, MFDataset
import pandas as pd

In [2]:
#------
# MODEL INFO
#------
    
# load mesh
path_mesh = '/pscratch/sd/c/cnissen/'
file_mesh = 'Nissen2022_FESOM_REcoM_mesh_information_corrected_20220910.nc'

f1 = Dataset(path_mesh+file_mesh) #xr.open_dataset(path+file1)
lat      = f1.variables['lat'][:]
lon      = f1.variables['lon'][:]
zlevs    = f1.variables['zlevs'][:]
cavities = f1.variables['cavity'][:]
topo     = f1.variables['topo'][:]
area     = f1.variables['cell_area'][:]
volume   = f1.variables['cell_volume'][:]
f1.close()
print(lat.shape)
ind_cavities = np.where(cavities==1)[0]


(72411,)


In [3]:
#----
# FUNCTIONS
#----

# get subareas based on lat only
def get_subarea_indices_sectors_lat_only(lat,lon):
    #subareas  = ['Larsen/FRIS','eWeddell','Amery/Australian','Ross Sea','Amundsen/Bellingh']
    
    lat_threshold = -60 #-60 # 0=all SH data
        
    ross1 = 150 #160
    ross2 = -130 # -145
    
    eWeddell1 = -10
    
    print ('Subareas: south of '+str(-1*lat_threshold)+'°S...')
        
    shelf_ind = np.zeros(len(lon))
    for i in range(1,len(lon)+1):
        dd = lon[i-1]
        if lat[i-1]<lat_threshold and dd>-52 and dd<eWeddell1: # Filchner-Ronne & Larsen 
            shelf_ind[i-1]=1
        elif lat[i-1]<-74 and dd>-85 and dd<eWeddell1: # Filchner-Ronne & Larsen in the very south
            shelf_ind[i-1]=1
        elif lat[i-1]<lat_threshold and dd>eWeddell1 and dd<30: # eWeddell
            shelf_ind[i-1]=6
        elif lat[i-1]<lat_threshold and dd>30 and dd<90: # Prydz
            shelf_ind[i-1]=2
        elif lat[i-1]<lat_threshold and dd>90 and dd<ross1: # Amery & Australian
            shelf_ind[i-1]=3
        elif lat[i-1]<lat_threshold and dd>ross1:  # Ross Sea 
            shelf_ind[i-1]=4
        elif lat[i-1]<lat_threshold and dd<ross2:   # Ross Sea
            shelf_ind[i-1]=4
        elif lat[i-1]<lat_threshold and dd>ross2 and dd<-69: #dd<-66: # Amundsen & Bellinghausen
            shelf_ind[i-1]=5
        elif lat[i-1]<lat_threshold and dd>-69 and dd<-52: #dd<-66: # Antarctic Peninsula
            wap_lat = -70
            if lat[i-1]>wap_lat:
                shelf_ind[i-1]=7 # pensinsula
            else:
                #shelf_ind[i-1]=7 
                # now find correct separation between Amundsen & Weddell Sea
                if dd>-66:
                    shelf_ind[i-1]=1
                else:
                    shelf_ind[i-1]=5
                    
            
        # now find correct separation between Amundsen & Weddell Sea
        #if lat[i-1]<=wap_lat:
                       
 #       # now find correct separation between Amundsen & Weddell Sea
 #       if lat[i-1]<-66 and dd>-63 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-66 and lat[i-1]>-70 and dd>-64.5 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-66.8 and lat[i-1]>-70 and dd>-66 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-64 and lat[i-1]>-66 and dd>-60.6 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-64.8 and lat[i-1]>-66 and dd>-62.4 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-64.7 and lat[i-1]>-65.9 and dd>-62.2 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-63.4 and lat[i-1]>-64.5 and dd>-58 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
            
    return shelf_ind

In [4]:
#---
# plot map of subregions
#----

shelf_ind = get_subarea_indices_sectors_lat_only(lat,lon)

ind = np.where((topo>2000) & (cavities==0))[0]
shelf_ind[ind]=0
shelf_ind[cavities==1]=0

weddell = np.zeros_like(shelf_ind)
weddell[shelf_ind==1]=1

eWeddell = np.zeros_like(shelf_ind)
eWeddell[shelf_ind==6]=1

prydz = np.zeros_like(shelf_ind)
prydz[shelf_ind==2]=1

adelie = np.zeros_like(shelf_ind)
adelie[shelf_ind==3]=1

ross = np.zeros_like(shelf_ind)
ross[shelf_ind==4]=1

amundsen = np.zeros_like(shelf_ind)
amundsen[shelf_ind==5]=1

wap = np.zeros_like(shelf_ind)
wap[shelf_ind==7]=1

#----
# get masks for MPAs
#----

path1 = '/pscratch/sd/c/cnissen/HLRN_runs_postprocessed/masks/masks_MPAs/'

f1 = Dataset(path1+'MPA_mask_AntarcticPeninsula_fesom_mesh_COARZE.nc')
mask_WAP_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_East_Antarctica_1_fesom_mesh_COARZE.nc')
mask_eastAA1_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_East_Antarctica_2_fesom_mesh_COARZE.nc')
mask_eastAA2_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_East_Antarctica_3_fesom_mesh_COARZE.nc')
mask_eastAA3_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_Orkney_fesom_mesh_COARZE.nc')
mask_Orkney_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_RossSea_fesom_mesh_COARZE.nc')
mask_RossSea_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_WeddellSea_fesom_mesh_COARZE.nc')
mask_WeddellSea_MPA = f1.variables['mask_fesom'][:]
f1.close()

#------------

subareas  = ['Larsen/FRIS','eWeddell','Amery/Australian','Ross Sea','Amundsen/Bellingh',\
             'south of SR4','south of 60S']
subareas = ['south of SR4']
# ADD "south_of_SR4" and "south_of_60S"
# additionally, load masks for "south of SR4" and "south of 60
path_mask_SR4 = '/pscratch/sd/c/cnissen/HLRN_runs_postprocessed/masks/mask_south_of_transect_SR4.nc'
path_mask_60S = '/pscratch/sd/c/cnissen/HLRN_runs_postprocessed/masks/mask_south_of_transect_60S.nc'
path_mask_shelfBreak = '/pscratch/sd/c/cnissen/HLRN_runs_postprocessed/masks/mask_south_of_transect_shelf_break.nc'
fileSR4 = Dataset(path_mask_SR4)
file60S = Dataset(path_mask_60S)
file_shelfBreak = Dataset(path_mask_shelfBreak)
maskSR4 = fileSR4.variables['mask'][:]
mask60S = file60S.variables['mask'][:]
mask_shelfBreak = file_shelfBreak.variables['mask'][:]

print (np.min(shelf_ind),np.max(shelf_ind))
latlim = -60
depth_threshold = 2000

#-----
# plot map of where subareas are
#-----

from matplotlib.colors import ListedColormap

# make the color map:
cmap1 = ListedColormap(['white','darkblue','cornflowerblue','firebrick',\
                        'gold','orange','mediumpurple','violet','mediumturquoise'])
cmap1 = ListedColormap(['white',(51./255.,34./255.,136./255.),(136./255.,204./255.,238./255.),\
                  (68./255.,170./255.,153./255.),(17./255.,119./255.,51./255.),\
                  (153./255.,153./255.,51./255.),(221./255.,204./255.,119./255.),\
                   (204./255.,102./255.,119./255.),(136./255.,34./255.,85./255.)])

levels1 = np.arange(0,10,1)

plot_regions = False
if plot_regions:
    dpicnt = 200
    elem_no_nan = mesh.elem[mesh.no_cyclic_elem,:]

    fig8= plt.figure(num=18, figsize=(4,3), dpi=200, facecolor='w', edgecolor='k')
    map = Basemap(projection='spstere',boundinglat=-60,lon_0=-180,resolution='l')
    #map.drawcoastlines()
    map.drawcountries()#
    map.drawmapboundary(fill_color='0.7')
    x, y = map(mesh.x2, mesh.y2)
    shelf_ind[shelf_ind==0]=-999
    mm=plt.tricontourf(x, y, elem_no_nan[::], shelf_ind, levels = levels1, \
                        cmap=cmap1, extend='both') #,vmin=-4, vmax=4)
    cbar = plt.colorbar(orientation='horizontal',fraction=0.075, pad=0.03,shrink=0.75)# ,ticks=cticks)
    #cbar.cmap.set_under('white')
    cbar.remove() 
    #plt.draw()
    lw = 1.2

    topo_plot = np.copy(mesh.topo)
    topo_plot[cavities==1]=-999
    m2=plt.tricontour(x, y, elem_no_nan[::], topo_plot, levels = [2000],linewidths=[lw],colors=['black'])

    save_plot = False
    if save_plot:
        plt.savefig(savepath1+'Map_subareas_for_C_anth_analysis.png',\
                                            dpi = dpicnt, bbox_inches='tight') 
    plt.show()
    #plt.close(fig8)

# Weddell
# eWeddell
# Prydz
# Adelie
# Ross
# Amundsen/Bellinghausen
# Antarctic Pensinsula


Subareas: south of 60°S...
0.0 7.0


In [6]:
#-----
# load model data & extract bottom layer omega A, omega C, pH 
#-----
from tqdm import tqdm

which_sim = 'simAssp585'
path = '/pscratch/sd/c/cnissen/carbonate_chemistry_3D_v2/'+which_sim+'/' 

vari_list = ['omegaA_avg_bottom','omegaC_avg_bottom','pH_avg_bottom']

if which_sim in ['simAssp126','simAssp245','simAssp370','simCssp245']:
    year_list1 = np.arange(2015,2100+1,1) 
elif which_sim in ['simAssp585','simB','simCssp585']:
    year_list1 = np.arange(1990,2100+1,1) 

#subregions = ['all_shelves','WeddellSea','eWeddellSea','Prydz','Adelie','RossSea','Amundsen_Bellingshausen','WAP',\
#             'RossSea_MPA','RossSea_MPA_shelf','RossSea_MPA_openOcean',\
#              'WeddellSea_MPA','WeddellSea_MPA_shelf','WeddellSea_MPA_openOcean',\
#             'Orkney_MPA','AntarcticPeninsula_MPA',\
#             'East_Antarctica_1_MPA','East_Antarctica_2_MPA','East_Antarctica_3_MPA','East_Antarctica_all_MPA',\
#             'East_Antarctica_1_MPA_shelf','East_Antarctica_2_MPA_shelf',\
#              'East_Antarctica_3_MPA_shelf','East_Antarctica_all_MPA_shelf',\
#             'East_Antarctica_1_MPA_openOcean','East_Antarctica_2_MPA_openOcean',\
#              'East_Antarctica_3_MPA_openOcean','East_Antarctica_all_MPA_openOcean']

subregions = ['AntarcticPeninsula_MPA','AntarcticPeninsula_MPA_shelf','AntarcticPeninsula_MPA_openOcean']

omegaA_bottom_model1 = np.zeros([12,len(year_list1),len(subregions)])
omegaC_bottom_model1 = np.zeros([12,len(year_list1),len(subregions)])
pH_bottom_model1     = np.zeros([12,len(year_list1),len(subregions)])
for yy in range(0,len(year_list1)):
    print('Process year '+str(year_list1[yy])+' of '+which_sim)
    path  = '/pscratch/sd/c/cnissen/carbonate_chemistry_3D_v2/'+which_sim+'/'
    f1    = Dataset(path+'OmegaA_fesom_'+str(year_list1[yy])+'0101.nc') 
    f2    = Dataset(path+'OmegaC_fesom_'+str(year_list1[yy])+'0101.nc') 
    f3    = Dataset(path+'pH_fesom_'+str(year_list1[yy])+'0101.nc')
    
    for mm in tqdm(range(0,12)):
        
        data_mean1 = f1.variables['OmegaA'][mm,:,:]#.mean(axis=0)  # depth x mesh.n2d
        data_mean2 = f2.variables['OmegaC'][mm,:,:]#.mean(axis=0) 
        data_mean3 = f3.variables['pH'][mm,:,:]#.mean(axis=0) 
        
        #----
        # extract model data in bottom layer
        #----
        #print ("Extract bottom layer from model output...")
        omegaA_1 = np.zeros([len(lon)])
        omegaC_1 = np.zeros([len(lon)])
        pH_1     = np.zeros([len(lon)])
        for i in np.arange(1,len(lon)+1):  
        # double-check if this is correct with python indexing (didn't work without the "-1")
            if cavities[i-1]==0: # not of the regions are within cavity, so only process those nodes here
                if lat[i-1]<=59.0: # only process nodes in the SO
                    a1 = data_mean1[:,i-1]
                    a = a1.mask==False
                    b1 = np.where(a==True)[0]
                    if len(b1)>0:
                        omegaA_1[i-1] = a1[b1[-1]]
                    del a1,a,b1
                    a1 = data_mean2[:,i-1]
                    a = a1.mask==False
                    b1 = np.where(a==True)[0]
                    if len(b1)>0:
                        omegaC_1[i-1] = a1[b1[-1]]
                    del a1,a,b1
                    a1 = data_mean3[:,i-1]
                    a = a1.mask==False
                    b1 = np.where(a==True)[0]
                    if len(b1)>0:
                        pH_1[i-1] = a1[b1[-1]]
                    del a1,a,b1

        get_regional_avg = True
        if get_regional_avg:
            for rr in range(0,len(subregions)):
                #print subregions[rr]
                if subregions[rr] in ['all_shelves']:
                    ind_regions = np.where(shelf_ind>0)[0]
                elif subregions[rr] in ['WeddellSea']:
                    ind_regions = np.where(weddell==1)[0]
                elif subregions[rr] in ['eWeddellSea']:
                    ind_regions = np.where(eWeddell==1)[0]
                elif subregions[rr] in ['Prydz']:
                    ind_regions = np.where(prydz==1)[0]
                elif subregions[rr] in ['Adelie']:
                    ind_regions = np.where(adelie==1)[0]
                elif subregions[rr] in ['RossSea']:
                    ind_regions = np.where(ross==1)[0]
                elif subregions[rr] in ['Amundsen_Bellingshausen']:
                    ind_regions = np.where(amundsen==1)[0]
                elif subregions[rr] in ['WAP']:
                    ind_regions = np.where(wap==1)[0]

                elif subregions[rr] in ['RossSea_MPA']:
                    ind_regions = np.where((mask_RossSea_MPA==1) & (cavities==0))[0]
                elif subregions[rr] in ['RossSea_MPA_shelf']:
                    ind_regions = np.where((mask_RossSea_MPA==1) & (cavities==0) & (topo<=2000))[0]
                elif subregions[rr] in ['RossSea_MPA_openOcean']:
                    ind_regions = np.where((mask_RossSea_MPA==1) & (cavities==0) & (topo>2000))[0]
                elif subregions[rr] in ['WeddellSea_MPA']:
                    ind_regions = np.where((mask_WeddellSea_MPA==1) & (cavities==0))[0]
                elif subregions[rr] in ['WeddellSea_MPA_shelf']:
                    ind_regions = np.where((mask_WeddellSea_MPA==1) & (cavities==0) & (topo<=2000))[0]
                elif subregions[rr] in ['WeddellSea_MPA_openOcean']:
                    ind_regions = np.where((mask_WeddellSea_MPA==1) & (cavities==0) & (topo>2000))[0]
                elif subregions[rr] in ['AntarcticPeninsula_MPA']:
                    ind_regions = np.where((mask_WAP_MPA==1) & (cavities==0))[0]
                elif subregions[rr] in ['AntarcticPeninsula_MPA_shelf']:
                    ind_regions = np.where((mask_WAP_MPA==1) & (cavities==0) & (topo<=2000))[0]
                elif subregions[rr] in ['AntarcticPeninsula_MPA_openOcean']:
                    ind_regions = np.where((mask_WAP_MPA==1) & (cavities==0) & (topo>2000))[0]
                elif subregions[rr] in ['Orkney_MPA']:
                    ind_regions = np.where((mask_Orkney_MPA==1) & (cavities==0))[0]
                elif subregions[rr] in ['Orkney_MPA_shelf']:
                    ind_regions = np.where((mask_Orkney_MPA==1) & (cavities==0) & (topo<=2000))[0]
                elif subregions[rr] in ['Orkney_MPA_openOcean']:
                    ind_regions = np.where((mask_Orkney_MPA==1) & (cavities==0) & (topo>2000))[0]

                elif subregions[rr] in ['East_Antarctica_1_MPA']:
                    ind_regions = np.where((mask_eastAA1_MPA==1) & (cavities==0))[0]
                elif subregions[rr] in ['East_Antarctica_1_MPA_shelf']:
                    ind_regions = np.where((mask_eastAA1_MPA==1) & (cavities==0) & (topo<=2000))[0]
                elif subregions[rr] in ['East_Antarctica_1_MPA_openOcean']:
                    ind_regions = np.where((mask_eastAA1_MPA==1) & (cavities==0) & (topo>2000))[0]
                elif subregions[rr] in ['East_Antarctica_2_MPA']:
                    ind_regions = np.where((mask_eastAA2_MPA==1) & (cavities==0))[0]
                elif subregions[rr] in ['East_Antarctica_2_MPA_shelf']:
                    ind_regions = np.where((mask_eastAA2_MPA==1) & (cavities==0) & (topo<=2000))[0]
                elif subregions[rr] in ['East_Antarctica_2_MPA_openOcean']:
                    ind_regions = np.where((mask_eastAA2_MPA==1) & (cavities==0) & (topo>2000))[0]
                elif subregions[rr] in ['East_Antarctica_3_MPA']:
                    ind_regions = np.where((mask_eastAA3_MPA==1) & (cavities==0))[0]
                elif subregions[rr] in ['East_Antarctica_3_MPA_shelf']:
                    ind_regions = np.where((mask_eastAA3_MPA==1) & (cavities==0) & (topo<=2000))[0]
                elif subregions[rr] in ['East_Antarctica_3_MPA_openOcean']:
                    ind_regions = np.where((mask_eastAA3_MPA==1) & (cavities==0) & (topo>2000))[0]

                elif subregions[rr] in ['East_Antarctica_all_MPA']:
                    ind_regions = np.where(((mask_eastAA1_MPA==1) | (mask_eastAA2_MPA==1) | (mask_eastAA3_MPA==1) ) & (cavities==0))[0]
                elif subregions[rr] in ['East_Antarctica_all_MPA_shelf']:
                    ind_regions = np.where(((mask_eastAA1_MPA==1) | (mask_eastAA2_MPA==1) | (mask_eastAA3_MPA==1) ) & (cavities==0) & (topo<=2000))[0]
                elif subregions[rr] in ['East_Antarctica_all_MPA_openOcean']:
                    ind_regions = np.where(((mask_eastAA1_MPA==1) | (mask_eastAA2_MPA==1) | (mask_eastAA3_MPA==1) ) & (cavities==0) & (topo>2000))[0]

                omegaA_bottom_model1[mm,yy,rr] = np.sum(omegaA_1[ind_regions]*area[ind_regions])/np.sum(area[ind_regions])
                omegaC_bottom_model1[mm,yy,rr] = np.sum(omegaC_1[ind_regions]*area[ind_regions])/np.sum(area[ind_regions])
                pH_bottom_model1[mm,yy,rr]     = np.sum(pH_1[ind_regions]*area[ind_regions])/np.sum(area[ind_regions])
                
        del data_mean1,data_mean2,data_mean3
        del pH_1,omegaA_1,omegaC_1
        
    f1.close()
    f2.close()
    f3.close()
    
    #-----
    # save as netcdf file
    #-----
    
    save_netcdf = True
    # only save every year (takes too much time otherwise)
    if save_netcdf:
        savepath = '/pscratch/sd/c/cnissen/carbonate_chemistry_3D_v2/regional/'
        netcdf_name = 'Carbonate_chemistry_regional_'+which_sim+'_'+str(year_list1[0])+'_'+str(year_list1[-1])+'_v4_AntarcticPeninsula_only.nc'

        if not os.path.exists(savepath+netcdf_name):
            print('Create file '+savepath+netcdf_name)
            w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
            w_nc_fid.regions1 = '1=AntarcticPeninsula_MPA, 2=AntarcticPeninsula_MPA_shelf, 3=AntarcticPeninsula_MPA_openOcean'
                       
            w_nc_fid.mocsy_source = 'https://github.com/jamesorr/mocsy'
            w_nc_fid.mocsy_dir    = '/home/ollie/ncara/scripts/mocsy/mocsy'
            w_nc_fid.script    = '/global/homes/c/cnissen/scripts/plot_PAPER_carbonate_chemistry_regional_save_netcdf_v3.ipynb'
            # create dimension & variable
            w_nc_fid.createDimension('month', 12)  # monthly
            w_nc_fid.createDimension('year', len(year_list1)) 
            w_nc_fid.createDimension('subregions', len(subregions)) 
            w_nc_fid.createDimension('nodes_2d', len(topo)) 
            w_nc_fid.createVariable('year', 'f4',('year'))
            w_nc_fid.variables['year'][:] = year_list1

            w_nc_fid.createVariable('mask_subregions', 'f4',('nodes_2d'))
            w_nc_fid.variables['mask_subregions'][:] = shelf_ind

            for vv in range(0,len(vari_list)):
                w_nc_var1 = w_nc_fid.createVariable(vari_list[vv], 'f4',('month','year','subregions'))
            w_nc_fid.close()
            
        else: # file exists, but check if variables exist as well
            w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC') 
            for vv in range(0,len(vari_list)):
                try:
                    w_nc_var1 = w_nc_fid.createVariable(vari_list[vv], 'f4',('month','year','subregions'))
                except: 
                    if yy==0:
                        print (vari_list[vv]+' exists in file')
                    pass
            w_nc_fid.close()
                    
        for vv in range(0,len(vari_list)):
            w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC') 
            if vari_list[vv] in ['omegaA_avg_bottom']:
                w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_bottom_model1[:,yy,:]
            elif vari_list[vv] in ['omegaC_avg_bottom']:
                w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_bottom_model1[:,yy,:]
            elif vari_list[vv] in ['pH_avg_bottom']:
                w_nc_fid.variables[vari_list[vv]][:,yy,:] = pH_bottom_model1[:,yy,:]
                    
            w_nc_fid.close()  
        del netcdf_name
                
print ('done')


Process year 1990 of simAssp585


100%|██████████| 12/12 [00:16<00:00,  1.39s/it]


Process year 1991 of simAssp585


100%|██████████| 12/12 [00:17<00:00,  1.43s/it]


Process year 1992 of simAssp585


...

Process year 2099 of simAssp585


100%|██████████| 12/12 [00:16<00:00,  1.41s/it]


Process year 2100 of simAssp585


100%|██████████| 12/12 [00:18<00:00,  1.54s/it]

done
